In [7]:
import cv2
import glob
import numpy as np
import os
import random
import tensorflow as tf

In [28]:
img = np.array(Image.open("dog.jpeg"))
h, w = img.shape[:2]
min_size = min(h, w)
h_new = (h/min_size) * 299
w_new = (w/min_size) * 299

resized = tf.image.resize_images(img, (h_new, w_new)).numpy().astype(np.uint8)
Image.fromarray(resized).save("resized.jpeg")

In [30]:
class DataGenerator:
    
    def __init__(self, train_dir=None, test_dir=None, shuffle_train=False, validation_split=0.0, 
                 n_frames=16):
        self.n_frames = n_frames
        self.frame_h = 224
        self.frame_w = 224
        self.classes = ["kickflip", "heelflip"]
        self.n_classes = len(self.classes)
        self.label_encoder = preprocessing.LabelEncoder()
        self.label_encoder.fit(self.classes)
        
        if train_dir:
            pathname = os.path.join(train_dir, "*")
            filepaths = glob.glob(pathname)
            if shuffle_train:
                random.shuffle(filepaths)
            
            n_train = int((1.0-validation_split) * len(filepaths))
            self.train_filepaths = filepaths[:n_train]
            self.val_filepaths = filepaths[n_train:]
            
    def get_train_generator(self):
        while True:
            for train_filepath in self.train_filepaths:
                frames = self._get_frames(train_filepath)
                label = self._get_label(train_filepath)
                labels_encoded = self.label_encoder.transform([label])
                labels_one_hot = to_categorical(labels_encoded, 
                                                num_classes=self.n_classes).astype(int)

                yield np.array([frames]), labels_one_hot

    def _get_frames(self, filepath):
        video = cv2.VideoCapture(filepath)
        if not video.isOpened():
            raise FileNotFoundError("The input video path you provided is invalid.")
        
        frames = []
        while video.isOpened():
            grabbed, frame_bgr = video.read()
            if not grabbed:
                break
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        video.release()
        
        if len(frames) < self.n_frames:
            n_pad_frames = self.n_frames - len(frames)
            for _ in range(n_pad_frames):
                blank_frame = np.zeros((self.frame_h, self.frame_w, 3))
                frames.append(blank_frame)
        
        frames_processed = []
        current_ix = 0
        step_size = len(frames) / float(self.n_frames)
        for _ in range(self.n_frames):
            frame = cv2.resize(frames[int(current_ix)], (self.frame_h, self.frame_w))
            frames_processed.append(frame)
            current_ix += step_size
        
        return frames_processed

    def _get_label(self, filepath):
        file_name = filepath.split("/")[-1]
        return file_name.split("-")[0]

In [31]:
dg = DataGenerator(train_dir="train")
train_gen = dg.get_train_generator()